# Import librairies

In [1]:
# To check PEP8 need installation : pip install flake8 pycodestyle pycodestyle_magic
# add %%flake8 in each block to check it
%load_ext pycodestyle_magic

In [195]:
# %%flake8

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.dummy import DummyClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, KFold
from sklearn.metrics import fbeta_score, make_scorer

# Load data files

In [4]:
# %%flake8

data_train = pd.read_csv('data_train.csv')

# Overview of data

In [15]:
# To optimize display of the Dataframe

#increase max rows displayed to 999
pd.set_option('display.max_rows', 999)

#increase max columns displayed to 999
pd.set_option('display.max_columns', 999)
#increase columns width to 100
pd.set_option('display.max_colwidth', 200)

# Formating float
pd.options.display.float_format = '{:.2f}'.format

In [6]:
# %%flake8

# define random_state
rd_state = 16

**Train data**

In [7]:
data_train.head(10)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATIO

In [14]:
data_train.describe(include='all')

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATIO

In [18]:
data_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 245 columns):
 #    Column                                             Non-Null Count   Dtype  
---   ------                                             --------------   -----  
 0    SK_ID_CURR                                         307511 non-null  int64  
 1    NAME_CONTRACT_TYPE                                 307511 non-null  int64  
 2    FLAG_OWN_CAR                                       307511 non-null  int64  
 3    FLAG_OWN_REALTY                                    307511 non-null  int64  
 4    CNT_CHILDREN                                       307511 non-null  int64  
 5    AMT_INCOME_TOTAL                                   307511 non-null  float64
 6    AMT_CREDIT                                         307511 non-null  float64
 7    AMT_ANNUITY                                        307499 non-null  float64
 8    AMT_GOODS_PRICE                                    307233 non-

In [19]:
print('data_train number of rows : {}' .format(data_train.shape[0]))
print('data_train number of columns : {}' .format(data_train.shape[1]))

data_train number of rows : 307511
data_train number of columns : 245


# Modelization test

## Split train / test data

In [47]:
# %%flake8

# split data between features and target
X = data_train.drop(columns=['TARGET'])
y = data_train['TARGET']

# Feature names
features = list(data_train.columns)

In [50]:
# %%flake8

# Separation training / testing set use stratify due to classes are imbalanced
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=rd_state, stratify=y)

In [65]:
# %%flake8

# sampling X_train
X_sample_train, X_sample_test, y_sample_train, y_sample_test = \
    train_test_split(
        X_train, y_train, train_size=0.2,
        random_state=rd_state, stratify=y_train)

## Testing model

In [70]:
# %%flake8

# dictionnary of score
# create a fbeta score
f2_scorer = make_scorer(fbeta_score, beta=2)
scores = {'AUC': 'roc_auc', 'fbeta': f2_scorer,
          'precision_recall_curve': 'average_precision'}

In [ ]:
#define cv
cv=KFold(5,shuffle=True,random_state=0)

In [130]:
# %%flake8

def test_model(X_train, y_train, steps, params):

    ''' create function to test different model '''

    pipeline = imbpipeline(steps=steps)

    grid = GridSearchCV(
        pipeline, param_grid=params, cv=cv, return_train_score=True,
        scoring=scores, refit='precision_recall_curve', n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid

### Dumy classifier

In [176]:
# %%flake8

# Median imputation of missing values
imputer = SimpleImputer(strategy='median')

# Normalize values
scaler = StandardScaler()

# class are imbalanced
# oversampling of the minority class
over = SMOTE(random_state=rd_state)

# undersampling of the majority class
under = RandomUnderSampler(random_state=rd_state)

# classifier to test
classifier = DummyClassifier(random_state=rd_state)

# steps for the pipeline
steps_dummyclassifier = [['imputer', imputer], ['scaler', scaler],
                         ['over', over], ['under', under],
                         ['classifier', classifier]]

# parameters to test
params_dummyclassifier = {}

# use function test_model
grid_dummyclassifier = test_model(
    X_sample_train, y_sample_train, steps, params)

In [153]:
# %%flake8

def display_result(grid, model):
    ''' create function to display dataframe of the results '''
    results = pd.DataFrame(grid.cv_results_)
    results.insert(0, 'Model', model)
    display(results)
    return results

In [158]:
# %%flake8

# use function display_result to display dataframe
# of the result for DummyClassifier
results_dummyclassifier = display_result(
    gris_dummyclassifier, 'DummyClassifier')

,Model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_AUC,split1_test_AUC,split2_test_AUC,split3_test_AUC,split4_test_AUC,mean_test_AUC,std_test_AUC,rank_test_AUC,split0_train_AUC,split1_train_AUC,split2_train_AUC,split3_train_AUC,split4_train_AUC,mean_train_AUC,std_train_AUC,split0_test_fbeta,split1_test_fbeta,split2_test_fbeta,split3_test_fbeta,split4_test_fbeta,mean_test_fbeta,std_test_fbeta,rank_test_fbeta,split0_train_fbeta,split1_train_fbeta,split2_train_fbeta,split3_train_fbeta,split4_train_fbeta,mean_train_fbeta,std_train_fbeta,split0_test_precision_recall_curve,split1_test_precision_recall_curve,split2_test_precision_recall_curve,split3_test_precision_recall_curve,split4_test_precision_recall_curve,mean_test_precision_recall_curve,std_test_precision_recall_curve,rank_test_precision_recall_curve,split0_train_precision_recall_curve,split1_train_precision_recall_curve,split2_train_precision_recall_curve,split3_train_precision_recall_curve,split4_train_precision_recall_curve,mean_train_precision_recall_curve,std_train_precision_recall_curve
0,DummyClassifier,3.59,0.55,0.59,0.08,{},0.50,0.50,0.50,0.50,0.50,0.50,0.00,1,0.50,0.50,0.50,0.50,0.50,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,0.08,0.08,0.08,0.08,0.08,0.00,1,0.08,0.08,0.08,0.08,0.08,0.08,0.00


In [150]:
# %%flake8

def display_score(result, grid):
    ''' create function to display score and best params for a model '''

    print('Fit time: %.2f' % result.loc[grid.best_index_]['mean_fit_time'])
    print('AUC score: %.3f' % result.loc[grid.best_index_]['mean_test_AUC'])
    print('fbeta score: %.3f' % result.loc[grid.best_index_]
          ['mean_test_fbeta'])
    print('precision_recall_curve score: %.3f' % grid.best_score_)
    print('Best parameters: %s' % grid.best_params_)

In [159]:
# %%flake8
# use function display_score to display score and
# best_params for DummyClassifier
display_score(results_dummyclassifier, grid_dummyclassifier)

Fit time: 3.59
AUC score: 0.500
fbeta score: 0.000
precision_recall_curve score: 0.081
Best parameters: {}


### Logistic Regression

In [173]:
# %%flake8

# Median imputation of missing values
imputer = SimpleImputer(strategy='median')

# Normalize values
scaler = StandardScaler()

# class are imbalanced
# oversampling of the minority class
over = SMOTE(random_state=rd_state)

# undersampling of the majority class
under = RandomUnderSampler(random_state=rd_state)

# classifier to test
classifier = LogisticRegression(random_state=rd_state, max_iter=10000)

# steps for the pipeline
steps_logisticregression = [['imputer', imputer], ['scaler', scaler],
                            ['over', over], ['under', under],
                            ['classifier', classifier]]

# parameters to test
params_logisticregression = {'classifier__C': [0.1, 1, 10, 100],
                             'over__sampling_strategy': [0.1, 0.2, 0.3],
                             'under__sampling_strategy': [0.4, 0.5, 0.6]}

# use function test_model
grid_logisticregression = test_model(
    X_sample_train, y_sample_train, steps_logisticregression,
    params_logisticregression)

In [165]:
# %%flake8

# use function display_result to display
# dataframe of the result for LogisticRegression
results_logisticregression = display_result(
    grid_logisticregression, 'LogisticRegression')

,Model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_over__sampling_strategy,param_under__sampling_strategy,params,split0_test_AUC,split1_test_AUC,split2_test_AUC,split3_test_AUC,split4_test_AUC,mean_test_AUC,std_test_AUC,rank_test_AUC,split0_train_AUC,split1_train_AUC,split2_train_AUC,split3_train_AUC,split4_train_AUC,mean_train_AUC,std_train_AUC,split0_test_fbeta,split1_test_fbeta,split2_test_fbeta,split3_test_fbeta,split4_test_fbeta,mean_test_fbeta,std_test_fbeta,rank_test_fbeta,split0_train_fbeta,split1_train_fbeta,split2_train_fbeta,split3_train_fbeta,split4_train_fbeta,mean_train_fbeta,std_train_fbeta,split0_test_precision_recall_curve,split1_test_precision_recall_curve,split2_test_precision_recall_curve,split3_test_precision_recall_curve,split4_test_precision_recall_curve,mean_test_precision_recall_curve,std_test_precision_recall_curve,rank_test_precision_recall_curve,split0_train_precision_recall_curve,split1_train_precision_recall_curve,split2_train_precision_recall_curve,split3_train_precision_recall_curve,split4_train_precision_recall_curve,mean_train_precision_recall_curve,std_train_precision_recall_curve
0,LogisticRegression,5.76,0.12,0.70,0.01,0.10,0.10,0.40,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.4}",0.72,0.74,0.73,0.74,0.75,0.74,0.01,1,0.76,0.75,0.76,0.75,0.75,0.75,0.00,0.30,0.30,0.27,0.32,0.31,0.30,0.02,36,0.33,0.32,0.33,0.32,0.31,0.32,0.01,0.21,0.20,0.20,0.22,0.22,0.21,0.01,5,0.23,0.23,0.23,0.23,0.23,0.23,0.00
1,LogisticRegression,5.64,0.16,0.63,0.03,0.10,0.10,0.50,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.5}",0.72,0.74,0.73,0.74,0.74,0.73,0.01,4,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.34,0.35,0.32,0.37,0.36,0.35,0.02,14,0.37,0.37,0.37,0.37,0.36,0.37,0.00,0.20,0.20,0.20,0.22,0.22,0.21,0.01,26,0.23,0.22,0.23,0.23,0.22,0.23,0.00
2,LogisticRegression,5.18,0.25,0.73,0.11,0.10,0.10,0.60,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.6}",0.72,0.74,0.73,0.74,0.74,0.73,0.01,6,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.37,0.38,0.35,0.38,0.39,0.38,0.01,3,0.40,0.39,0.39,0.39,0.39,0.39,0.00,0.20,0.20,0.20,0.22,0.22,0.21,0.01,30,0.23,0.22,0.22,0.22,0.22,0.22,0.00
3,LogisticRegression,7.39,0.28,0.69,0.05,0.10,0.20,0.40,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.4}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,13,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.31,0.31,0.29,0.32,0.33,0.31,0.01,28,0.33,0.32,0.33,0.33,0.32,0.33,0.00,0.21,0.20,0.20,0.22,0.22,0.21,0.01,2,0.23,0.23,0.23,0.22,0.22,0.23,0.00
4,LogisticRegression,7.06,0.15,0.66,0.06,0.10,0.20,0.50,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.5}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,15,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.35,0.34,0.33,0.36,0.36,0.35,0.01,15,0.37,0.36,0.37,0.36,0.36,0.37,0.00,0.21,0.20,0.20,0.22,0.22,0.21,0.01,7,0.23,0.23,0.23,0.22,0.22,0.23,0.00
5,LogisticRegression,6.25,0.13,0.70,0.02,0.10,0.20,0.60,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.6}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,14,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.37,0.37,0.35,0.38,0.38,0.37,0.01,5,0.39,0.39,0.39,0.39,0.39,0.39,0.00,0.21,0.20,0.20,0.22,0.22,0.21,0.01,18,0.23,0.23,0.22,0.22,0.22,0.22,0.00
6,LogisticRegression,9.21,0.51,0.68,0.05,0.10,0.30,0.40,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.3, 'under__sampling_strategy': 0.4}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,25,0.75,0.75,0.75,0.75,0.75,0.75,0.00,0.31,0.31,0.28,0.32,0.32,0.31,0.01,32,0.34,0.32,0.33,0.33,0.33,0.33,0.00,0.21,0.21,0.20,0.22,0.22,0.21,0.01,6,0.23,0.23,0.23,0.22,0.22,0.22,0.00
7,LogisticRegression,8.78,0.46,0.77,0.10,0.10,0.30,0.50,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.3, 'under__sampling_strategy': 0.5}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,26,0.75,0.75,0.75,0.75,0.75,0.75,0.00,0.35,0.34,0.32,0.35,0.36,0.34,0.01,24,0.37,0.36,0.37,0.36,0.37

In [187]:
# %%flake8

# use function display_score to display score and
# best_params for LogisticRegression
display_score(results_logisticregression, grid_logisticregression)

Fit time: 10.85
AUC score: 0.731
fbeta score: 0.311
precision_recall_curve score: 0.211
Best parameters: {'classifier__C': 1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.4}


### Random Forest Classifier

In [188]:
# %%flake8

# Median imputation of missing values
imputer = SimpleImputer(strategy='median')

# Normalize values
scaler = StandardScaler()

# class are imbalanced
# oversampling of the minority class
over = SMOTE(random_state=rd_state)

# undersampling of the majority class
under = RandomUnderSampler(random_state=rd_state)

# classifier to test
classifier = RandomForestClassifier(random_state=rd_state)

# steps for the pipeline
steps_randomforestclassifier = [['imputer', imputer], ['scaler', scaler],
                                      ['over', over], ['under', under],
                                      ['classifier', classifier]]

# parameters to test
params_randomforestclassifier = {
    'classifier__max_depth': np.arange(10, 50, 5),
    'classifier__min_samples_split': np.arange(2, 20, 2),
    'classifier__bootstrap': [True],
    'over__sampling_strategy': [0.1, 0.2, 0.3],
    'under__sampling_strategy': [0.4, 0.5, 0.6]}

# use function test_model
grid_randomforestclassifier = test_model(
    X_sample_train, y_sample_train, steps_randomforestclassifier,
    params_randomforestclassifier)

In [189]:
# %%flake8

# use function display_result to display
# dataframe of the result for RandomForestClassifier
results_randomforestclassifier = display_result(
    grid_randomforestclassifier, 'RandomForestClassifier')

,Model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__bootstrap,param_classifier__max_depth,param_classifier__min_samples_split,param_over__sampling_strategy,param_under__sampling_strategy,params,split0_test_AUC,split1_test_AUC,split2_test_AUC,split3_test_AUC,split4_test_AUC,mean_test_AUC,std_test_AUC,rank_test_AUC,split0_train_AUC,split1_train_AUC,split2_train_AUC,split3_train_AUC,split4_train_AUC,mean_train_AUC,std_train_AUC,split0_test_fbeta,split1_test_fbeta,split2_test_fbeta,split3_test_fbeta,split4_test_fbeta,mean_test_fbeta,std_test_fbeta,rank_test_fbeta,split0_train_fbeta,split1_train_fbeta,split2_train_fbeta,split3_train_fbeta,split4_train_fbeta,mean_train_fbeta,std_train_fbeta,split0_test_precision_recall_curve,split1_test_precision_recall_curve,split2_test_precision_recall_curve,split3_test_precision_recall_curve,split4_test_precision_recall_curve,mean_test_precision_recall_curve,std_test_precision_recall_curve,rank_test_precision_recall_curve,split0_train_precision_recall_curve,split1_train_precision_recall_curve,split2_train_precision_recall_curve,split3_train_precision_recall_curve,split4_train_precision_recall_curve,mean_train_precision_recall_curve,std_train_precision_recall_curve
0,RandomForestClassifier,7.33,0.05,1.17,0.09,True,10,2,0.10,0.40,"{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.4}",0.72,0.74,0.72,0.73,0.73,0.73,0.01,14,0.89,0.88,0.89,0.88,0.88,0.88,0.00,0.08,0.07,0.06,0.09,0.07,0.07,0.01,285,0.22,0.24,0.24,0.23,0.20,0.23,0.01,0.21,0.20,0.20,0.21,0.21,0.21,0.01,18,0.47,0.47,0.47,0.46,0.47,0.47,0.00
1,RandomForestClassifier,7.26,0.29,1.04,0.06,True,10,2,0.10,0.50,"{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.5}",0.72,0.74,0.72,0.73,0.73,0.73,0.01,47,0.88,0.88,0.88,0.88,0.88,0.88,0.00,0.18,0.19,0.14,0.21,0.18,0.18,0.02,137,0.40,0.39,0.38,0.39,0.36,0.39,0.01,0.21,0.20,0.20,0.21,0.22,0.21,0.01,15,0.46,0.43,0.46,0.44,0.44,0.44,0.01
2,RandomForestClassifier,6.92,0.41,1.02,0.10,True,10,2,0.10,0.60,"{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.6}",0.72,0.74,0.72,0.72,0.74,0.73,0.01,18,0.88,0.88,0.88,0.87,0.88,0.88,0.00,0.26,0.29,0.24,0.29,0.26,0.27,0.02,66,0.49,0.50,0.50,0.49,0.47,0.49,0.01,0.20,0.20,0.20,0.21,0.21,0.20,0.01,34,0.43,0.43,0.44,0.43,0.43,0.43,0.00
3,RandomForestClassifier,11.65,0.18,1.05,0.09,True,10,2,0.20,0.40,"{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.4}",0.71,0.72,0.70,0.71,0.72,0.71,0.01,372,0.88,0.88,0.88,0.87,0.88,0.88,0.00,0.01,0.00,0.02,0.01,0.02,0.01,0.01,575,0.04,0.04,0.05,0.03,0.04,0.04,0.01,0.19,0.18,0.19,0.18,0.19,0.19,0.00,510,0.43,0.43,0.41,0.42,0.42,0.42,0.01
4,RandomForestClassifier,10.90,0.13,1.26,0.18,True,10,2,0.20,0.50,"{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.5}",0.71,0.72,0.70,0.71,0.72,0.71,0.01,403,0.88,0.88,0.87,0.87,0.87,0.87,0.00,0.04,0.04,0.05,0.05,0.05,0.04,0.01,399,0.12,0.14,0.15,0.14,0.11,0.13,0.01,0.19,0.19,0.18,0.19,0.19,0.19,0.00,438,0.40,0.42,0.40,0.41,0.41,0.41,0.01
5,RandomForestClassifier,9.92,0.60,1.02,0.14,True,10,2,0.20,0.60,"{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.6}",0.71,0.72,0.70,0.71,0.72,0.71,0.01,399,0.88,0.87,0.87,0.87,0.87,0.87,0.00,0.13,0.10,0.11,0.14,0.11,0.12,0.01,246,0.28,0.26,0.30,0.27,0.25,0.27,0.02,0.19,0.18,0.18,0.19,0.19,0.18,0.00,550,0.38,0.39,0.39,0.37,0.39,0.39,0.01
6,RandomForestClassifier,15.97,0.44,1.14,0.18,True,10,

In [190]:
# %%flake8

# use function display_score to display score and
# best_params for RandomForestClassifier
display_score(results_randomforestclassifier, grid_randomforestclassifier)

Fit time: 6.57
AUC score: 0.729
fbeta score: 0.077
precision_recall_curve score: 0.209
Best parameters: {'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_split': 14, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.4}


### Hist Gradient Boosting Classifier

In [ ]:
# %%flake8

# Median imputation of missing values
imputer = SimpleImputer(strategy='median')

# Normalize values
scaler = StandardScaler()

# class are imbalanced
# oversampling of the minority class
over = SMOTE(random_state=rd_state)

# undersampling of the majority class
under = RandomUnderSampler(random_state=rd_state)

# classifier to test
classifier = HistGradientBoostingClassifier(random_state=rd_state)

# steps for the pipeline
steps_histgradientboostingclassifier = [['imputer', imputer], ['scaler', scaler],
                                      ['over', over], ['under', under],
                                      ['classifier', classifier]]

# parameters to test
params_histgradientboostingclassifier = {
    'classifier__loss': ['least_squares','least_absolute_deviation'],
    'classifier__learning_rate': np.arange(0.05,0.2, 0.05),
    'classifier__max_iter':np.arange(100,400, 100),
    'classifier__max_depth': np.arange(10, 100, 10),
    'over__sampling_strategy': [0.1, 0.2, 0.3],
    'under__sampling_strategy': [0.4, 0.5, 0.6]}

# use function test_model
grid_histgradientboostingclassifier = test_model(
    X_sample_train, y_sample_train, steps_histgradientboostingclassifier,
    params_histgradientboostingclassifier)

In [ ]:
# %%flake8

# use function display_result to display
# dataframe of the result for GradientBoostingClassifier
results_histgradientboostingclassifier = display_result(
    grid_histgradientboostingclassifier, 'HistRandomForestClassifier')

In [ ]:
# %%flake8

# use function display_score to display score and
# best_params for GradientBoostingClassifier
display_score(results_histgradientboostingclassifier, grid_histgradientboostingclassifier)